In [1]:
# pip install langchain-google-genai 
# pip install langchain

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
google_gemini_api=os.getenv("GOOGLE_API_KEY")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm_model=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=google_gemini_api)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1721647494.562697  164691 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721647494.568280  164691 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721647494.569374  164691 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
# TEMPLATE0: Generate a well-researched paragraph on a disease
TEMPLATE0="""
You are an expert in medicine, specializing in the human body and various medical fields that affect human health. Your extensive knowledge encompasses a wide range of diseases, treatments, and medications. 
Your task is to write a unique, 300-word paragraph on {disease_name} . Paragraph should be well-researched, informative, and distinct from previous ones, highlighting the causes, symptoms, treatments, and any other relevant information about the disease.
"""

In [7]:
para_generation_prompt = PromptTemplate(
    input_variables=["disease_name"],
    template=TEMPLATE0
)

In [8]:
para_chain=LLMChain(llm=llm_model, prompt=para_generation_prompt, output_key="para", verbose=True)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
TEMPLATE1="""
Text:{para}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for the disease {disease_name} for creating the public awareness. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["para", "number", "disease_name", "response_json"],
    template=TEMPLATE1
)

In [11]:
quiz_chain=LLMChain(llm=llm_model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [12]:
TEMPLATE2="""
You are an medical expert and health advocates or public health advocates. Given a Multiple Choice Quiz for {disease_name} disease.\
You need to evaluate the quality of the question and give a complete analysis of the quiz. Only use at max 50 words for quality analysis. 
if the quiz is not at per with the quality and awreness and helpfull related to the public,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the public awareness and health
Quiz_MCQs:
{quiz}

Check from an medical expert and health advocates or public health advocates of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["disease_name", "quiz"], template=TEMPLATE2)

In [14]:
review_chain=LLMChain(llm=llm_model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(
    chains=[para_chain,quiz_chain, review_chain], 
    input_variables=["disease_name", "number", "response_json"],
    output_variables=["para", "quiz", "review"], 
    verbose=True,
)

## loading the data for generating the MCQs

In [16]:
import pandas as pd
import json

In [17]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
MCQ_NUMBERS=5
DISEASE_NAME="Maleria"

In [19]:
response=generate_evaluate_chain(
    {
        "number":MCQ_NUMBERS,
        "disease_name":DISEASE_NAME,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert in medicine, specializing in the human body and various medical fields that affect human health. Your extensive knowledge encompasses a wide range of diseases, treatments, and medications. 
Your task is to write a unique, 300-word paragraph on Maleria . Paragraph should be well-researched, informative, and distinct from previous ones, highlighting the causes, symptoms, treatments, and any other relevant information about the disease.



I0000 00:00:1721647495.810325  164832 subchannel.cc:806] subchannel 0x12f3353e0 {address=ipv6:%5B2404:6800:4002:82e::200a%5D:443, args={grpc.client_channel_factory=0x11f26b370, grpc.default_authority=generativelanguage.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x11f26b350, grpc.internal.client_channel_call_destination=0x11e7bba18, grpc.internal.event_engine=0x118c0e4a0, grpc.internal.security_connector=0x11b004680, grpc.internal.subchannel_pool=0x11f270900, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.1, grpc.resource_quota=0x11f26cc80, grpc.server_uri=dns:///generativelanguage.googleapis.com:443}}: connect failed (UNKNOWN:connect: No route to host (65) {created_time:"2024-07-22T16:54:55.807616+05:30"}), backing off for 1000 ms



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text:Malaria, a mosquito-borne parasitic infection, remains a formidable public health threat, particularly in tropical and subtropical regions. Caused by Plasmodium parasites, malaria is transmitted through the bite of infected female Anopheles mosquitoes. Symptoms typically manifest within 10-15 days after a bite and may include fever, chills, sweating, headache, muscle pain, nausea, and vomiting. In severe cases, malaria can lead to life-threatening complications such as cerebral malaria, respiratory distress, and organ failure. Diagnosis involves microscopy or rapid diagnostic tests to detect the parasites in the blood. Treatment options include artemisinin-based combination therapies (ACTs), which have significantly reduced malaria-related mortality. Preventive measures include the use of insecticide-treated mosquito nets, insect repellents, and antimalarial prophylaxis for travelers visiting endemic a

In [20]:
response

{'number': 5,
 'disease_name': 'Maleria',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'para': 'Malaria, a mosquito-borne parasitic infection, remains a formidable public health threat, particularly in tropical and subtropical regions. Caused by Plasmodium parasites, malaria is transmitted through the bite of infected female Anopheles mosquitoes. Symptoms typically manifest within 10-15 days after a bite and may include fever, chills, sweating, headache, muscle pain, nausea, and vomiting. In severe cases, malaria can lead to

In [21]:
new_quiz_df=pd.DataFrame([response])

In [22]:
new_quiz_df

,number,disease_name,response_json,para,quiz,review
0,5,Maleria,"{""1"": {""mcq"": ""multiple choice question"", ""opt...","Malaria, a mosquito-borne parasitic infection,...","### RESPONSE_JSON\n{""1"": {""mcq"": ""What is the ...",**Quality Analysis:**\n\nThe quiz covers essen...


In [23]:
para_json=new_quiz_df.loc[0,'para'].strip()

In [24]:
para_json

'Malaria, a mosquito-borne parasitic infection, remains a formidable public health threat, particularly in tropical and subtropical regions. Caused by Plasmodium parasites, malaria is transmitted through the bite of infected female Anopheles mosquitoes. Symptoms typically manifest within 10-15 days after a bite and may include fever, chills, sweating, headache, muscle pain, nausea, and vomiting. In severe cases, malaria can lead to life-threatening complications such as cerebral malaria, respiratory distress, and organ failure. Diagnosis involves microscopy or rapid diagnostic tests to detect the parasites in the blood. Treatment options include artemisinin-based combination therapies (ACTs), which have significantly reduced malaria-related mortality. Preventive measures include the use of insecticide-treated mosquito nets, insect repellents, and antimalarial prophylaxis for travelers visiting endemic areas. Ongoing research focuses on developing more effective vaccines, diagnostic too

In [25]:
quiz_json = new_quiz_df.loc[0, 'quiz'].split('### RESPONSE_JSON\n')[1]
quiz_data = json.loads(quiz_json)

# Display the quiz questions
for key, value in quiz_data.items():
    print(f"Question {key}: {value['mcq']}")
    for option_key, option_value in value['options'].items():
        print(f"  {option_key}: {option_value}")
    print(f"Correct Answer: {value['correct']}\n")

Question 1: What is the primary mode of transmission for malaria?
  a: Contact with contaminated water
  b: Bite of infected mosquitoes
  c: Inhalation of spores
  d: Ingestion of contaminated food
Correct Answer: b

Question 2: Which of the following is a common symptom of malaria?
  a: Diarrhea
  b: Muscle pain
  c: Skin rash
  d: Sneezing
Correct Answer: b

Question 3: What is the recommended first-line treatment for malaria?
  a: Quinine
  b: Chloroquine
  c: Artemisinin-based combination therapies (ACTs)
  d: Tetracycline
Correct Answer: c

Question 4: Which of the following is a preventive measure against malaria?
  a: Vaccination
  b: Use of insecticide-treated mosquito nets
  c: Regular blood transfusions
  d: Avoidance of travel to endemic areas
Correct Answer: b

Question 5: What is a potential life-threatening complication of severe malaria?
  a: Anemia
  b: Cerebral malaria
  c: Pneumonia
  d: Gastrointestinal bleeding
Correct Answer: b



In [26]:
quiz_list = []
for key, value in quiz_data.items():
    question = value['mcq']
    options = value['options']
    correct_ans = value['correct']
    row = {
        'question': question,
        'optiona': options.get('a', ''),
        'optionb': options.get('b', ''),
        'optionc': options.get('c', ''),
        'optiond': options.get('d', ''),
        'correctans': correct_ans
    }
    quiz_list.append(row)

# Convert to DataFrame
df_quiz = pd.DataFrame(quiz_list)

# Save to CSV
df_quiz.to_csv('quiz_questions.csv', index=False)